## Purpose

The aim of this study is to make a model which can accurately predict the medical insurance charges of a person based on their: <br>
- age: age of primary beneficiary
- bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9
- sex: insurance contractor gender, female, male
- children: Number of children covered by health insurance / Number of dependents
- smoker: Smoking
- region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.
- charges: Individual medical costs billed by health insurancee.nkwuranceurance

## Imports

In [28]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

## Data exploration

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [30]:
df=pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [31]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

We check for missing values because they can negatively impact model performance.Some models do not handle missing values well, leading to inaccurate predictions. Luckily there are not NA values, a good day :)


In [32]:
X=df.drop("charges",axis=1)
y=df["charges"]

## Data pre-processing pipeline

### Seperation of the categorical and numerical columns

In [33]:
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols =X.select_dtypes(include=["object"]).columns


The code snippet allows us to seperate the columns containing categorical and numerical columns very easily

### Preprocessor pipelines

In [34]:
preprocessor=ColumnTransformer([
    ("num", StandardScaler(),numerical_cols),
    ("cat", OneHotEncoder(drop='first',sparse_output=False), categorical_cols),
])

preprocessor_poly=ColumnTransformer([
    ("num",Pipeline(steps=[("poly", PolynomialFeatures(degree=2,include_bias=False)),("num",StandardScaler()), ]), numerical_cols),
    ("cat", OneHotEncoder(drop='first', sparse_output=False), categorical_cols),])

ColumnTransformer helps us handle different data types (numerical and categorical) in a single pipeline efficiently.
- StandardScaler normalizes numerical features to a common scale, which improves model convergence and performance.
-  OneHotEncoder transforms categorical features into variables suitable forlinear regression model.
    -  We use 'drop='first'' to avoid the dummy variable trap

### Train, test split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Finding the best performing model

In [36]:
names=["lin_reg","ridge_regression","lasso_regression","polynomial_features","bagging_regressor", "xgboost_regressor"]
models=[LinearRegression(),
        Ridge(alpha=5.0),
        Lasso(alpha=5.0),
        LinearRegression(),
        BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=3), n_estimators=10,  ),
        XGBRegressor(n_estimators=10,max_depth=3,)]

dict_of_models=dict(zip(names,models))
def give_best(dict_of_models, X_train, X_test,y_train,y_test):
  scores=[]
  for model in dict_of_models.keys():
      pipeline_func=Pipeline(steps=[
          ("preprocess", preprocessor) if model!="polynomial_features" else ("preprocess",preprocessor_poly),
          (model,dict_of_models[model ])
      ])
      pipeline_func.fit( X_train,y_train)
      y_predict= pipeline_func.predict(X_test)
      rmse = np.sqrt(mean_squared_error(y_test, y_predict))
      scores.append(rmse)
      print(f"{model} RMSE:{rmse}")

  return list(dict_of_models.values())[scores.index(min(scores))]

best_model=give_best(dict_of_models, X_train, X_test,y_train,y_test)
print(f"#####################################################")
print(f"the best performing model is:")
best_model

lin_reg RMSE:5796.284659276275
ridge_regression RMSE:5821.574140942559
lasso_regression RMSE:5800.079215603098
polynomial_features RMSE:5841.28018590557
bagging_regressor RMSE:4445.24162178276
xgboost_regressor RMSE:4294.171968597672
#####################################################
the best performing model is:


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Then **give_best** iterates through different models defined in the 'dict_of_models'dictionary. It builds pipelines that include preprocessing and the specific model for every model defined in the *dict_of_models* . The pipeline is then used to fit  the training data and used to make predictions on the testing data. Then, RMSE (Root Mean Squared Error) is calculated to evaluate the model's performance on the testing set. The function returns the best performing model based on the lowest RMSE score.

## Optimizing the best performing model using GridSearchCV

In [37]:
param_grid = {
    'xgboost_regressor__n_estimators': [10, 50, 100],
    'xgboost_regressor__max_depth': [3, 5, 10],
    'xgboost_regressor__eta': [0.3, 0.1, 0.01],
    'xgboost_regressor__subsample': [0.5, 0.7, 1.0],
    "xgboost_regressor__colsample_bytree":[0.5, 0.7, 1.0],
    "xgboost_regressor__reg_lambda":[0.5, 0.7, 1.0],
}
pipeline_to_grid_search=Pipeline(steps=[
    ("preprocess", preprocessor),
    ("xgboost_regressor",best_model )
])
grid_search = GridSearchCV(pipeline_to_grid_search,param_grid,cv=6, scoring='neg_mean_squared_error', n_jobs=-1,verbose=3)
grid_search.fit(X_train,y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best RMSE (negative mean squared error):", np.sqrt(-grid_search.best_score_))

y_test_predict = grid_search.best_estimator_.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
print(f"Test RMSE: {rmse_test}")


Fitting 6 folds for each of 729 candidates, totalling 4374 fits
Best Parameters: {'xgboost_regressor__colsample_bytree': 1.0, 'xgboost_regressor__eta': 0.1, 'xgboost_regressor__max_depth': 3, 'xgboost_regressor__n_estimators': 50, 'xgboost_regressor__reg_lambda': 1.0, 'xgboost_regressor__subsample': 1.0}
Best RMSE (negative mean squared error): 4576.613260512437
Test RMSE: 4246.596096089942


## For experimentation

### Linear regression

In [38]:
pipeline=Pipeline(steps=[
    ("preprocess",preprocessor ),
    ("lin_reg", LinearRegression()),

])

pipeline.fit(X_train, y_train)
y_predict=pipeline.predict(X_train)
mse=mean_squared_error(y_train, y_predict)
rmse=np.sqrt(mse)
rmse


6105.545160099847

### Polynomial Regression

In [39]:
pipeline_poly=Pipeline(steps=[
    ("preprocess",preprocessor_poly ),
    ("lin_reg", LinearRegression()),

])
pipeline_poly.fit(X_train, y_train)
y_predict=pipeline_poly.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test,y_predict))
rmse

5841.28018590557

### Using Random Forest

In [40]:
base_reg = DecisionTreeRegressor(max_depth=3)
pipeline_bagging = Pipeline( steps=[
    ("preprocess", preprocessor),
    ("bagging_regressor", BaggingRegressor(estimator=base_reg, n_estimators=10,  ))
])
pipeline_bagging.fit(X_train, y_train)
y_predict=pipeline_bagging.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test,y_predict))
rmse

4595.250603287242

### Using XGBoost

In [41]:
pipeline_xgboost = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("xgboost_regressor", XGBRegressor(n_estimators=10,max_depth=3,))
])

pipeline_xgboost.fit(X_train, y_train)
y_predict = pipeline_xgboost.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_predict))
print(rmse)


4294.171968597672
